In [1]:
import nltk
from nltk.wsd import lesk
#from supwsd.wsd import SupWSD
from collections import defaultdict
import sys
sys.path.append("../")
from itertools import product
from collections import defaultdict
import json
import pytrips

In [2]:
from pytrips.ontology import load
ont = load()

no synset found for distinct%5:00:00::
no synset found for distinct%5:00:00::
no synset found for absolute%3:00:00:inalienable:00
no synset found for absolute%3:00:00:inalienable:00
no synset found for be%2:42:012::
no synset found for be%2:42:012::
no synset found for favourite%3:00:00:popular:00
no synset found for favourite%3:00:00:loved:00
no synset found for favourite%3:00:00:popular:00
no synset found for favourite%3:00:00:loved:00
no synset found for flavor%2:35:00::
no synset found for flavor%2:35:00::
no synset found for ideal%3:00:00:perfect:00
no synset found for ideal%3:00:00:perfect:00
no synset found for immoral%3:00:00:wrong:01
no synset found for immoral%3:00:00:wrong:01
no synset found for another%3:00:00::
no synset found for another%3:00:00::
no synset found for nauseous%3:00:00:ill:01
no synset found for nauseated%3:00:00:ill:01
no synset found for queasy%3:00:00:ill:01
no synset found for sickish%3:00:00:ill:01
no synset found for nauseous%3:00:00:ill:01
no synset 

In [3]:
from IPython.display import clear_output, Image, display

In [3]:
# def load_data(fname):
#     return json.load(open(fname))

# ## Load lex and ontology from json files
# ont_data = load_data("../data/ontology.json")
# lex_data = load_data("../data/lexicon.json")

# #Store the ontology in a dictionary with nodes as keys
# ont_dict = {}
# for item in ont_data:
#     ont_dict[item['name']] = item
    
# #Load TRIPS-types
# ont = load()

In [6]:
senses = ['determine', 'becoming-aware', 'scrutiny']
senses = [x.upper() for x in senses]
senses

['DETERMINE', 'BECOMING-AWARE', 'SCRUTINY']

In [7]:
for sense in senses:
    display(ont[sense].path_to_root())

[ont::determine,
 ont::cogitation,
 ont::change-awareness,
 ont::event-of-awareness,
 ont::event-of-change,
 ont::situation-root,
 ont::referential-sem,
 ont::any-sem,
 ont::root]

[ont::becoming-aware,
 ont::acquire-belief,
 ont::cogitation,
 ont::change-awareness,
 ont::event-of-awareness,
 ont::event-of-change,
 ont::situation-root,
 ont::referential-sem,
 ont::any-sem,
 ont::root]

[ont::scrutiny,
 ont::cogitation,
 ont::change-awareness,
 ont::event-of-awareness,
 ont::event-of-change,
 ont::situation-root,
 ont::referential-sem,
 ont::any-sem,
 ont::root]

In [8]:
ont['cogitation'].children

[ont::learn,
 ont::predict,
 ont::scrutiny,
 ont::acquire-belief,
 ont::determine,
 ont::pay-attention]

In [20]:
# def add_scores(senses):
#     '''
#     Input: A list of trips-ontology types (strings)
    
#     Output: 1. add sense-score to ont TRIPS-types starting bottom-up (no output)
    
#             2. a set of nodes being considered for the word senses
#     '''
#     sense_list = []
    
#     for sense in senses:
#         path_to_root = ont[sense].path_to_root()
#         for trip_type in path_to_root:
#             if trip_type not in sense_list:
#                 sense_list.append(trip_type)
#             if trip_type.sense_score:
#                 trip_type.sense_score += 1
#             else:
#                 trip_type.sense_score = 1
                
#     return sense_list

# def reset_scores():
#     '''
#     Reset all sense scores in the ontology
#     '''
#     for key in ont_dict:
#         ont[key].reset_score()
       
#     return None

def sense_closure(senses):
    '''
    Input: A list of trips-ontology types (strings)
            
    Output: A list of trips-ontology types (TRIPS-type)
    
            For each trips-ont type in the input, we build the tree going up to the
            root of TRIPS-ontology and add every ontology appearing in the way to our
            final output list
    
    '''
    sense_list = []
    
    for sense in senses:
        path_to_root = ont[sense].path_to_root()
        for trip_type in path_to_root:
            if trip_type not in sense_list:
                sense_list.append(trip_type)
    
    return sense_list
                
def break_in_branch(children, senses_considered ):
    '''
    Input: 1. children: a list of TRIPS-types - children of a particular node
            2. senses_considered: a list of TRIPS-types - all trips-types
                                    being considered for current word-sense
    
    Output: Boolean
            if more than 1 child is present in senses_considered
    '''
    count=0
    for child in children:
        if count>1:
            return True
        if child in senses_considered:
            count+=1
    if count==1:
        return False
    if count==0:
        print("No sense present in the children")
    
def top_disam_nodes(senses_considered):
    '''
    Input: A list of trips-types being considered for disambiguating 
            word-senses
            
    Output: A list of highest disambiguating trips-type
    '''
    #init_score = ont['root'].sense_score
    current_node = ont['root']
    
    ans = []
    
    while True:
        current_children = current_node.children
        #print(current_children)
        if break_in_branch(current_children, senses_considered):
            for child in current_children:
                if child in senses_considered:
                    ans.append(child)
            return ans
        else:
            for child in current_children:
                if child in senses_considered:
                    current_node = child

In [21]:
senses

['DETERMINE', 'BECOMING-AWARE', 'SCRUTINY']

In [22]:
senses_considered = sense_closure(senses)
senses_considered

[ont::determine,
 ont::cogitation,
 ont::change-awareness,
 ont::event-of-awareness,
 ont::event-of-change,
 ont::situation-root,
 ont::referential-sem,
 ont::any-sem,
 ont::root,
 ont::becoming-aware,
 ont::acquire-belief,
 ont::scrutiny]

In [23]:
top_disam_nodes(senses_considered)

[ont::scrutiny, ont::acquire-belief, ont::determine]

In [24]:
"I hear bass sounds"

'I hear bass sounds'

### Rough

In [6]:
"The car hit the bus."

'The car hit the bus.'

In [26]:
trips_probs = [('ont::touch', 0.11150409427676958),
  ('ont::change', 0.08906610379073356),
  ('ont::arrive', 0.0874618767894344),
  ('ont::evoke-injury', 0.08635276717242853),
  ('ont::hitting', 0.0859967736282921),
  ('ont::affect-experiencer', 0.08546717884883379),
  ('ont::attack', 0.08452608782703594),
  ('ont::acquire', 0.08259924765722312),
  ('ont::collide', 0.061849488043740436),
  ('ont::complete', 0.0422583443948868)]

In [27]:
trips_probs = [('ont::geo-formation', 0.25479920680573653),
  ('ont::assets', 0.1705654240774585),
  ('ont::mountain', 0.16348259882408828),
  ('ont::collection', 0.16013912161918406),
  ('ont::organization-abstr', 0.12550682433676627),
  ('ont::organization', 0.12550682433676627)]

In [28]:
trips_senses = [x[5:] for x,y in trips_probs]
print(trips_senses)

['geo-formation', 'assets', 'mountain', 'collection', 'organization-abstr', 'organization']


In [31]:
senses_considered = sense_closure(trips_senses)
print(senses_considered)

[ont::geo-formation, ont::geo-object, ont::phys-object, ont::referential-sem, ont::any-sem, ont::root, ont::assets, ont::function-object, ont::tangible-abstract-object, ont::abstract-object, ont::mountain, ont::collection, ont::group-object, ont::organization-abstr, ont::social-group-abstr, ont::group-object-abstr, ont::abstract-object-nontemporal, ont::organization, ont::social-group]


In [32]:
top_disam_nodes(senses_considered)

[ont::group-object, ont::geo-object]

In [40]:
ont['organization-abstr'].path_to_root()

[ont::organization-abstr,
 ont::social-group-abstr,
 ont::group-object-abstr,
 ont::abstract-object-nontemporal,
 ont::abstract-object,
 ont::referential-sem,
 ont::any-sem,
 ont::root]